In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import defaultdict
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
from scipy.sparse import hstack
import sklearn.metrics as metrics
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
clean_dataset_path = "/content/drive/MyDrive/AIHW/AS2/caius_data_clean.csv"
df = None
if not os.path.exists(clean_dataset_path):
  df = None
  print("Empty Dataset")
else:
  df = pd.read_csv(clean_dataset_path)

In [ ]:
average_height = df['height'].dropna().mean()
df['height'] = df['height'].fillna(average_height)
average_rating = df['rating'].dropna().mean()
df['rating'] = df['rating'].fillna(average_rating)

In [ ]:
scaler = StandardScaler()
numerical_data = scaler.fit_transform(df[['size','height']])
fit_mapping = {'small': 0, 'fit': 1, 'large': 2}
y = df['fit'].map(fit_mapping).values
X = numerical_data

In [ ]:
over = SMOTE(sampling_strategy={0: int(len(y) * 0.5), 2: int(len(y) * 0.5)}, k_neighbors=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
# Define the pipeline
pipeline = Pipeline(steps=[('o', over)])
# Apply the pipeline to your data
X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [ ]:
# Create the SVM model
model = svm.SVC(kernel='rbf',verbose = True, max_iter = 1500,C = 2)

# Train the model
model.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(C=2, max_iter=1500, verbose=True)

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
decision_scores = model.decision_function(X_test)

In [ ]:
# Binarize the output
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))

# Initialize variables to store the cumulative AUC
auc_sum = 0
n_classes = y_test_binarized.shape[1]

# Compute ROC AUC for each class
for i in range(n_classes):
    # Compute ROC curve and AUC for this class
    fpr, tpr, _ = metrics.roc_curve(y_test_binarized[:, i], decision_scores[:, i])
    roc_auc = metrics.auc(fpr, tpr)
    auc_sum += roc_auc
    # Optionally, you can plot the ROC curves here

# Compute the average AUC
average_auc = auc_sum / n_classes
print(f"Average AUC: {average_auc}")

Average AUC: 0.47553581854899446


In [ ]:
decision_scores

array([[ 0.98528101,  2.11313494, -0.10620352],
       [ 1.01732331,  2.11053253, -0.11840763],
       [ 1.05312132,  2.15066652, -0.16785739],
       ...,
       [ 1.02298986,  2.0862763 , -0.09913497],
       [ 1.02215736,  2.10968868, -0.11988592],
       [ 1.00992608,  2.10614236, -0.11079758]])

In [ ]:
auc_score =  roc_auc_score(y_test, decision_scores,multi_class = 'ovr')

ValueError: ignored

In [ ]:
# Compute AUC
probabilities = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, probabilities)

AxisError: ignored

In [ ]:
report = classification_report(y_test, y_pred, target_names=['Small', 'Fit', 'Large'])

print(report)

              precision    recall  f1-score   support

       Small       0.18      0.29      0.22      2616
         Fit       0.74      0.50      0.60     14107
       Large       0.17      0.36      0.23      2532

    accuracy                           0.46     19255
   macro avg       0.36      0.39      0.35     19255
weighted avg       0.59      0.46      0.50     19255



In [ ]:
y_test.shape

(38509,)